## Hugging Face CLI

In [10]:
import getpass
print("Enter you Hugging Face token:")
TOKEN = getpass.getpass()

Enter you Hugging Face token:
··········


In [11]:
!git config --global credential.helper store
!huggingface-cli login --token $TOKEN --add-to-git-credential

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful




## Import the modules


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [13]:
# Suppressing “INFO” and “WARNING” messages by setting the verbosity of the Transformers library.
from transformers import logging
logging.set_verbosity_error()

# Suppressing Python warnings
import warnings
warnings.filterwarnings("ignore")

## Load the model with 8-bit Quantization

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

In [15]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quantized_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config = bnb_config,
                    device_map = "auto")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

## Inference of the pre-trained model

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
input = tokenizer("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?", return_tensors="pt").to('cuda')

response = quantized_model.generate(**input, max_new_tokens = 100)
print(tokenizer.batch_decode(response, skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? 48 + 24 = 72.\n## Step 1: Determine the number of clips Natalia sold in April.\nNatalia sold clips to 48 of her friends in April.\n\n## Step 2: Calculate the number of clips Natalia sold in May.\nShe sold half as many clips in May, so she sold 48 / 2 = 24 clips in May.\n\n## Step 3: Add the number of clips sold in April and May to find the total.\n48']


## Preprocessing Dataset

In [17]:
from datasets import load_dataset

dataset = "openai/gsm8k"
data = load_dataset(dataset, 'main')

tokenizer.pad_token = tokenizer.eos_token
data = data.map(lambda samples: tokenizer(samples["question"], samples["answer"], truncation=True, padding="max_length", max_length=100), batched=True)
train_samples = data["train"].select(range(400))

display(train_samples)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 400
})

In [18]:
print(train_samples[:1])

{'question': ['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'], 'answer': ['Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'], 'input_ids': [[128000, 45, 4306, 689, 6216, 27203, 311, 220, 2166, 315, 1077, 4885, 304, 5936, 11, 323, 1243, 1364, 6216, 4376, 439, 1690, 27203, 304, 3297, 13, 2650, 1690, 27203, 1550, 42701, 689, 4662, 31155, 304, 5936, 323, 3297, 30, 128000, 45, 4306, 689, 6216, 220, 2166, 14, 17, 284, 1134, 2166, 14, 17, 28, 1187, 2511, 1187, 27203, 304, 3297, 627, 45, 4306, 689, 6216, 220, 2166, 10, 1187, 284, 1134, 2166, 10, 1187, 28, 5332, 2511, 5332, 27203, 31155, 304, 5936, 323, 3297, 627, 827, 220, 5332, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Training the model on dataset

### LoRA configurations

In [19]:
import peft
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

### Setting the training arguments






In [20]:
from transformers import TrainingArguments
import os

working_dir = './'
output_directory = os.path.join(working_dir, "lora")

training_args = TrainingArguments(
    output_dir = output_directory,
    auto_find_batch_size = True,
    learning_rate = 3e-4,
    num_train_epochs=5
)

### Setting the trainer



In [21]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model = quantized_model,
    args = training_args,
    train_dataset = train_samples,
    peft_config = lora_config, tokenizer = tokenizer,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

### Training the model

It takes significant amount of time to train the model and we have a limited session time of Jupyter notebook on our platform. You can uncomment the code and execute on GPU enable machine to see the response.

We have already trained the model for you. We will load the saved model later in the code for inference of fine-tuned model.

In [ ]:
#trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
500,1.273000
1000,0.862500


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1000, training_loss=1.06775, metrics={'train_runtime': 2209.425, 'train_samples_per_second': 0.905, 'train_steps_per_second': 0.453, 'total_flos': 9014088499200000.0, 'train_loss': 1.06775, 'epoch': 5.0})

In [22]:
# Save the model.
model_path = os.path.join(output_directory, f"lora_model")

trainer.model.save_pretrained(model_path)

In [29]:
#We are going to clean some variables to avoid memory problems
import gc
import torch
del quantized_model
del trainer
del train_samples
del data
torch.cuda.empty_cache()
gc.collect()

0

## Load the fine-tuned model

In [30]:
model_path = "/trained_models/lora/lora_model"

from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

loaded_model = AutoPeftModelForCausalLM.from_pretrained(
                                        model_path,
                                        quantization_config = bnb_config,
                                        device_map = 'auto')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Inference of the fine-tuned model




In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
input = tokenizer("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?", return_tensors="pt").to('cuda')

response = loaded_model.generate(**input, max_new_tokens = 100)
print(tokenizer.batch_decode(response, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?The number of clips Natalia sold in May is 48/2 = <<48/2=24>>24 clips.\nThe total number of clips Natalia sold in April and May is 48+24 = <<48+24=72>>72 clips.\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n####']
